In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Button, Layout, Output, HBox
from IPython.display import Image, YouTubeVideo
from pathlib import Path
import numpy as np
from bqplot import OrdinalScale, LinearScale, Bars, Lines, Axis, Figure
from bqplot import pyplot as plt
import warnings

warnings.filterwarnings('ignore')


In [2]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
def create_expanded_output(description, button_style):
    return Output(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))


# Felix's dice race!

## Roll either one or two dice and track their progress as they race up the chart. Which number will win?

* Select how many dice you want to roll and how many times you need to roll a number for that number to win.
* To roll the dice hit the "Roll dice" button.
* Add the dice up (easy for one dice) - and press the correct orange button. Try to get a thumbs up!
* Add the dice roll to the race by pressing one of the "add to race" buttons at the bottom of the grid. You can skip adding up the dice (pressing the orange buttons) if you like.
* You can clear the dice roll by pressing the "Clear Dice" button. This is optional.
* To end the current race and start a new one, click "Clear Race".
* If playing on a phone it's better in landscape (-:

In [3]:
imgs = Path("images")
die = ['a']*6
die[0] = imgs/"one.png"
die[1] = imgs/"two.png"
die[2] = imgs/"three.png"
die[3] = imgs/"four.png"
die[4] = imgs/"five.png"
die[5] = imgs/"six.png"
thumbs_up = imgs/"thumbsup.png"
thumbs_down = imgs/"thumbsdown.png"

In [4]:
btn_one = Button(description='Roll one dice')
btn_two = Button(description='Roll two dice')
total = Output()
pause = Output()
pause.value = False
tracker_dict = {0: Output(),
               1: Output(),
               2: Output(),
               3: Output(),
               4: Output(),
               5: Output(),
               6: Output(),
               7: Output(),
               8: Output(),
               9: Output(),
               10: Output(),
               11: Output(),
               12: Output()}
for value in tracker_dict.values():
    value.value = 0

In [5]:
no_dice = widgets.RadioButtons(
    options=['one dice','two dice'],
    value='two dice', # Defaults to 'two dice'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Dice:',
    disabled=False
    )
winning_no = widgets.RadioButtons(
    options=[5,10,20,50],
    value=10, # Defaults to 10
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Race to:',
    disabled=False
    )

HBox([no_dice, winning_no])

In [6]:
grid = GridspecLayout(9,10, width='1000px', height='450px')

grid[0,:2] = create_expanded_button('Clear Dice', 'primary')
grid[0,2:4] = create_expanded_button('', 'primary')
grid[0,4:6] = create_expanded_button('Clear Race', 'danger')
grid[1:5,0:2] = create_expanded_button('Roll Dice', 'Success')
grid[1:5,2:4] = create_expanded_output('', 'info')
grid[1:5,4:6] = create_expanded_output('', 'info')
grid[7:,2:4] = create_expanded_output('result','')
grid[7:,:2] = create_expanded_button('add to race', 'primary')
grid[7:,4:6] = create_expanded_button('add to race', 'primary')
grid[:9,6:] = Output(layout=Layout(width='auto', height='auto'))
for i in range(2):
    for j in range(6):
        grid[i+5,j] = create_expanded_button('{}'.format(i*6 + (j + 1)), 'warning')

grid

GridspecLayout(children=(Button(button_style='primary', description='Clear Dice', layout=Layout(grid_area='wid…

In [7]:
chart = Output(height = 600, width = 300)
chart

Output()

In [8]:
def on_click_throw_dice(change):
    grid[1,2].clear_output()
    grid[1,4].clear_output()
    grid[7,3].clear_output()
    if no_dice.value == 'one dice':
        dice_throw = np.random.randint(6, size=None)
        with grid[1,2]: display(Image(die[dice_throw], width=195, height=195))
        # random ints are in (0, 5) representing dice throws in (1,6) - so add one!
        total.value = dice_throw+1
    if no_dice.value == 'two dice':
        dice_throw = np.random.randint(6, size=2)
        with grid[1,2]: display(Image(die[dice_throw[0]], width=195, height=195))
        with grid[1,4]: display(Image(die[dice_throw[1]], width=195, height=195))
        # random ints are in (0, 5) representing dice throws in (1,6) - so add two!
        total.value = dice_throw.sum()+2

grid[1,1].on_click(on_click_throw_dice)

In [9]:
def on_click_clear(change):
    grid[1,2].clear_output()
    grid[1,4].clear_output()
    grid[7,3].clear_output()
    
grid[0,0].on_click(on_click_clear)

In [10]:
def on_click_clear_all(change):
    grid[1,2].clear_output()
    grid[1,4].clear_output()
    grid[7,3].clear_output()
    grid[1,6].clear_output()
    chart.clear_output()
    for value in tracker_dict.values():
        value.value = 0
    
grid[0,5].on_click(on_click_clear_all)

In [11]:
def on_click_add(change):
    tracker_dict[total.value].value += 1
    chart.clear_output()
    grid[0,8].clear_output()
     
    if no_dice.value=='one dice': 
        x_min = 1
        x_max = 6
        x_data = np.asarray(list(np.arange(x_min,x_max+1)))
        y_data = np.array(np.zeros(len(x_data)),dtype=int)
        for i in range(x_max):
            y_data[i] = tracker_dict[i+1].value
    if no_dice.value=='two dice': 
        x_min = 2
        x_max = 12
        x_data = np.asarray(list(np.arange(x_min,x_max+1)))
        y_data = np.array(np.zeros(len(x_data)),dtype=int)
        for i in range(x_max-1):
            y_data[i] = tracker_dict[i+2].value
    
    
    x_ord = OrdinalScale()
    x_ord.min = x_min
    x_ord.max = x_max
    y_sc = LinearScale()
    y_sc.min = 0
    y_sc.max = winning_no.value

    bar = Bars(x=x_data, y=y_data, scales={'x': x_ord, 'y':y_sc})

    ax_x = Axis(scale=x_ord, grid_lines='none', 
                label='Dice Roll', label_offset='35px')
    ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='d',
                tick_values= np.arange(1,winning_no.value+1), 
                grid_lines='dashed', label='Number of rolls',
                label_offset='20px')

    race = Figure(marks=[bar], axes=[ax_x, ax_y], 
                  fig_margin = dict(top=60, bottom=80, left=60, right=30),
                  padding_x=0, padding_y=0, title='Race!!', title_offet='60px')
                  #layout=Layout(width='auto', height='auto'))
    
    with chart: display(race)
    with grid[0,8]: display(race)
        
    if max(y_data) == winning_no.value:
        grid[1,2].clear_output()
        grid[1,4].clear_output()
        with grid[1,2]: display(YouTubeVideo('by8Gt96pq58', height=195, width=195))
        with grid[1,4]: 
            if no_dice.value=='one dice': print(f'{(y_data.tolist().index(winning_no.value) + 1)} wins!!')
            if no_dice.value=='two dice': print(f'{(y_data.tolist().index(winning_no.value) + 2)} wins!!')
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
        for value in tracker_dict.values():
            value.value = 0
            
    
# def on_click_add(change):
#     chart_update()

grid[7,0].on_click(on_click_add)
grid[7,5].on_click(on_click_add)

In [18]:
# There must be a more effecient way to do this! Like return the location of the button that has been pressed
# Anyway, twelve functions to check whether the correct button was pressed or not.
def on_click_one(change):
    grid[7,3].clear_output()
    if total.value == 1: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,0].on_click(on_click_one)
# grid[5,0].on_click(on_click_add)

def on_click_two(change):
    grid[7,3].clear_output()
    if total.value == 2: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,1].on_click(on_click_two)
# grid[5,1].on_click(on_click_add)

def on_click_three(change):
    grid[7,3].clear_output()
    if total.value == 3: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,2].on_click(on_click_three)
# grid[5,2].on_click(on_click_add)

def on_click_four(change):
    grid[7,3].clear_output()
    if total.value == 4: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,3].on_click(on_click_four)
# grid[5,3].on_click(on_click_add)

def on_click_five(change):
    grid[7,3].clear_output()
    if total.value == 5: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,4].on_click(on_click_five)
# grid[5,4].on_click(on_click_add)

def on_click_six(change):
    grid[7,3].clear_output()
    if total.value == 6: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,5].on_click(on_click_six)
# grid[5,5].on_click(on_click_add)

def on_click_seven(change):
    grid[7,3].clear_output()
    if total.value == 7: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,0].on_click(on_click_seven)
# grid[6,0].on_click(on_click_add)

def on_click_eight(change):
    grid[7,3].clear_output()
    if total.value == 8: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,1].on_click(on_click_eight)
# grid[6,1].on_click(on_click_add)

def on_click_nine(change):
    grid[7,3].clear_output()
    if total.value == 9: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,2].on_click(on_click_nine)
# grid[6,2].on_click(on_click_add)

def on_click_ten(change):
    grid[7,3].clear_output()
    if total.value == 10: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,3].on_click(on_click_ten)
# grid[6,3].on_click(on_click_add)

def on_click_eleven(change):
    grid[7,3].clear_output()
    if total.value == 11: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,4].on_click(on_click_eleven)
# grid[6,4].on_click(on_click_add)

def on_click_twelve(change):
    grid[7,3].clear_output()
    if total.value == 12: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,5].on_click(on_click_twelve)
# grid[6,5].on_click(on_click_add)

In [13]:
####Experimenting with altair implementation of chart. It works, but not sure it improves it that much

# import pandas as pd
# index = range(1,13)
# data = [0] * 12
# dummy = data
# for i in range(1,13):
#     data[i-1] = tracker_dict[i].value

# data
# df=pd.DataFrame(np.zeros((12,2)),columns=['dice','quantity'])
# df.dice = range(1,13)
# df.quantity = data
# df

# mychart = alt.Chart(df).mark_bar().encode(
#     alt.X('dice:O'),
#     alt.Y('quantity:Q')
# )

In [14]:
# output = Output()
# with output: display(mychart)
# output